In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

# From DenseNets to Images
## Live Demos

In [2]:
fashion_data = pd.read_csv("data/fashion-mnist/fashion-mnist_train.csv")

In [3]:
fashion_data

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59995,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
59996,1,0,0,0,0,0,0,0,0,0,...,73,0,0,0,0,0,0,0,0,0
59997,8,0,0,0,0,0,0,0,0,0,...,160,162,163,135,94,0,0,0,0,0
59998,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
train_data = []
val_data = []
test_data = []
for group_label, data in fashion_data.groupby("label"):
    shuffled_data = data.sample(len(data))
    val_in_group = shuffled_data.iloc[:500]
    test_in_group = shuffled_data.iloc[500:1000]
    train_in_group = shuffled_data.iloc[1000:]
    
    train_data.append(train_in_group)
    val_data.append(val_in_group)
    test_data.append(test_in_group)

In [5]:
def merge_and_shuffle(datasets):
    result = pd.concat(datasets)
    return result.sample(len(result))

In [6]:
train_data = merge_and_shuffle(train_data)
val_data = merge_and_shuffle(val_data)
test_data = merge_and_shuffle(test_data)

In [7]:
train_data.shape, val_data.shape, test_data.shape

((50000, 785), (5000, 785), (5000, 785))

In [8]:
def split_label(dataset):
    return (dataset.drop("label", axis = 1).values, dataset.label.values)

In [9]:
train_attributes, train_labels = split_label(train_data)
val_attributes, val_labels = split_label(val_data)
test_attributes, test_labels = split_label(test_data)

In [10]:
def scale(dataset):
    return dataset.astype(float) / 255.0

In [11]:
train_attributes = scale(train_attributes)
val_attributes = scale(val_attributes)
test_attributes = scale(test_attributes)

In [12]:
N_INPUTS = train_attributes.shape[1]
N_CLASSES = len(set(train_labels))

N_INPUTS, N_CLASSES

(784, 10)

In [13]:
hyperparams = {
    "dropout_rate": 0.1,
    "batch_size": 16,
    "learning_rate": 0.005
}

In [14]:
neural_network = Sequential([
    Input(N_INPUTS),
    Dense(128, activation = "relu"),
    Dropout(hyperparams["dropout_rate"]),
    Dense(64, activation = "relu"),
    Dropout(hyperparams["dropout_rate"]),
    Dense(32, activation = "relu"),
    Dropout(hyperparams["dropout_rate"]),
    Dense(N_CLASSES, activation = "softmax")
])

In [15]:
neural_network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               100480    
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                2080      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                3

In [16]:
neural_network.compile(
    optimizer = Adam(learning_rate = hyperparams["learning_rate"]),
    loss = "sparse_categorical_crossentropy",
    metrics = ["accuracy"])

In [17]:
history = neural_network.fit(
    x = train_attributes,
    y = train_labels,
    validation_data = (val_attributes, val_labels),
    epochs = 20,
    batch_size = 16,
    callbacks = [TensorBoard()])

Train on 50000 samples, validate on 5000 samples
Epoch 1/20
50000/50000 [==============================] - 13s 253us/sample - loss: 0.6646 - accuracy: 0.7620 - val_loss: 0.5435 - val_accuracy: 0.8076
Epoch 2/20
50000/50000 [==============================] - 11s 220us/sample - loss: 0.5389 - accuracy: 0.8094 - val_loss: 0.4352 - val_accuracy: 0.8396
Epoch 3/20
50000/50000 [==============================] - 10s 204us/sample - loss: 0.5170 - accuracy: 0.8179 - val_loss: 0.4517 - val_accuracy: 0.8478
Epoch 4/20
50000/50000 [==============================] - 11s 215us/sample - loss: 0.5036 - accuracy: 0.8273 - val_loss: 0.4455 - val_accuracy: 0.8446
Epoch 5/20
50000/50000 [==============================] - 11s 222us/sample - loss: 0.4996 - accuracy: 0.8287 - val_loss: 0.4831 - val_accuracy: 0.8566
Epoch 6/20
50000/50000 [==============================] - 11s 211us/sample - loss: 0.4941 - accuracy: 0.8306 - val_loss: 0.4850 - val_accuracy: 0.8358
Epoch 7/20
50000/50000 [=====================

In [20]:
neural_network.evaluate(train_attributes, train_labels, verbose = 0)

[0.41394401565074923, 0.8567]

In [21]:
neural_network.evaluate(val_attributes, val_labels, verbose = 0)

[0.4719900562286377, 0.8484]

In [26]:
neural_network.predict_classes(val_attributes)

array([4, 8, 0, ..., 7, 6, 6], dtype=int64)